In [0]:
!kill -9 -1!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
        process = psutil.Process(os.getpid())
        print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize(           process.memory_info().rss))
print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


/bin/bash: line 0: kill: -1!ln: arguments must be process or job IDs
/bin/bash: line 0: kill: -sf: arguments must be process or job IDs
/bin/bash: line 0: kill: /opt/bin/nvidia-smi: arguments must be process or job IDs
/bin/bash: line 0: kill: /usr/bin/nvidia-smi: arguments must be process or job IDs
  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=8699c574962da7351fda24efdf44761f740d118fe28e19169fe3c8d27d560501
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB
Gen RAM Free: 12.8 GB  | Proc size: 158.9 MB


In [0]:
import numpy as np
import pandas as pd
import os
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from tensorflow import keras
layers = keras.layers
models = keras.models

In [0]:
from keras.datasets import reuters


Using TensorFlow backend.


In [0]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=5000)

2113536/2110848 [==============================] - 0s 0us/step


In [0]:
print("train_data ", train_data.shape)
print("train_labels ", train_labels.shape)

print("test_data ", test_data.shape)
print("test_labels ", test_labels.shape)

train_data  (8982,)
train_labels  (8982,)
test_data  (2246,)
test_labels  (2246,)


In [0]:
train_data

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]),
       list([1, 3267, 699, 3434, 2295, 56, 2, 2, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12]),
       list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110, 59

In [0]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [0]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

print("x_train ", x_train.shape)
print("x_test ", x_test.shape)

x_train  (8982, 10000)
x_test  (2246, 10000)


In [0]:
from keras.utils import to_categorical

In [0]:
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

print("one_hot_train_labels ", one_hot_train_labels.shape)
print("one_hot_test_labels ", one_hot_test_labels.shape)

one_hot_train_labels  (8982, 46)
one_hot_test_labels  (2246, 46)


In [0]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

print("x_val ", x_val.shape)
print("y_val ", y_val.shape)

print("partial_x_train ", partial_x_train.shape)
print("partial_y_train ", partial_y_train.shape)

x_val  (1000, 10000)
y_val  (1000, 46)
partial_x_train  (7982, 10000)
partial_y_train  (7982, 46)


In [0]:
from keras import regularizers

In [0]:
model = models.Sequential()

model.add(layers.Embedding(10000, 64))
model.add(layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(46, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 46)                2990      
Total params: 717,294
Trainable params: 717,294
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
batch_size = 16
epochs = 10
drop_ratio = 0.5

history = model.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

In [0]:
history = model.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
499/499 [==============================] - 282s 565ms/step - loss: 2.2460 - accuracy: 0.4236 - val_loss: 2.0557 - val_accuracy: 0.4640
Epoch 2/10
499/499 [==============================] - 281s 562ms/step - loss: 2.1254 - accuracy: 0.4481 - val_loss: 2.0806 - val_accuracy: 0.4570
Epoch 3/10
 21/499 [>.............................] - ETA: 4:04 - loss: 2.1177 - accuracy: 0.4435

KeyboardInterrupt: ignored

In [0]:
score = model.evaluate(x_test, one_hot_test_labels,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

71/71 [==============================] - 30s 419ms/step - loss: 2.4415 - accuracy: 0.3620
Test loss: 2.4415225982666016
Test accuracy: 0.36197686195373535


In [0]:
model2 = models.Sequential()

model2.add(layers.Embedding(5000, 64))
model2.add(layers.GRU(256, dropout=0.9, return_sequences=True))
model2.add(layers.GRU(256, dropout=0.9))
model2.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu'))

model2.add(layers.Dense(46, activation='softmax'))
model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          320000    
_________________________________________________________________
gru_4 (GRU)                  (None, None, 256)         247296    
_________________________________________________________________
gru_5 (GRU)                  (None, 256)               394752    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_11 (Dense)             (None, 46)                11822     
Total params: 1,039,662
Trainable params: 1,039,662
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model2.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
 41/250 [===>..........................] - ETA: 1:46 - loss: 5.4589 - accuracy: 0.3034

KeyboardInterrupt: ignored

In [0]:
score = model2.evaluate(x_test, one_hot_test_labels,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

71/71 [==============================] - 58s 815ms/step - loss: 2.0404 - accuracy: 0.4791
Test loss: 2.0403594970703125
Test accuracy: 0.4790739119052887


In [0]:
model3 = models.Sequential()
model3.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(10000,)))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu'))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(46, activation='softmax'))
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               2560256   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 46)                11822     
Total params: 2,637,870
Trainable params: 2,637,870
Non-trainable params: 0
_________________________________________________________________


In [0]:
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model3.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
250/250 [==============================] - 2s 7ms/step - loss: 5.5632 - accuracy: 0.5714 - val_loss: 2.9441 - val_accuracy: 0.6710
Epoch 2/10
250/250 [==============================] - 2s 6ms/step - loss: 2.8185 - accuracy: 0.6693 - val_loss: 2.5765 - val_accuracy: 0.7040
Epoch 3/10
250/250 [==============================] - 2s 6ms/step - loss: 2.6061 - accuracy: 0.6795 - val_loss: 2.4500 - val_accuracy: 0.6960
Epoch 4/10
250/250 [==============================] - 2s 6ms/step - loss: 2.5440 - accuracy: 0.6847 - val_loss: 2.4003 - val_accuracy: 0.6970
Epoch 5/10
250/250 [==============================] - 2s 7ms/step - loss: 2.5016 - accuracy: 0.6808 - val_loss: 2.3725 - val_accuracy: 0.6980
Epoch 6/10
250/250 [==============================] - 2s 7ms/step - loss: 2.4662 - accuracy: 0.6939 - val_loss: 2.3571 - val_accuracy: 0.7000
Epoch 7/10
250/250 [==============================] - 2s 7ms/step - loss: 2.4636 - accuracy: 0.6918 - val_loss: 2.3128 - val_accuracy: 0.7100
Epoch 

In [0]:
score = model3.evaluate(x_test, one_hot_test_labels,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

71/71 [==============================] - 0s 4ms/step - loss: 2.3972 - accuracy: 0.7088
Test loss: 2.3971922397613525
Test accuracy: 0.7088156938552856


In [0]:
model4 = models.Sequential()
model4.add(layers.Embedding(10000, 46,input_length = 60))
model4.add(layers.GRU(256, dropout=0.9, return_sequences=True))
model4.add(layers.GRU(256, dropout=0.9))
model4.add(layers.Dense(46, activation='sigmoid'))
model4.summary()
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 46)            460000    
_________________________________________________________________
gru_2 (GRU)                  (None, 60, 256)           233472    
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               394752    
_________________________________________________________________
dense_1 (Dense)              (None, 46)                11822     
Total params: 1,100,046
Trainable params: 1,100,046
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model4.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10


InternalError: ignored

In [0]:
model5 = models.Sequential()
model5.add(layers.Dense(512, input_shape=(5000,)))
model5.add(layers.Activation('relu'))
model5.add(layers.Dropout(0.5))
model5.add(layers.Dense(46))
model5.add(layers.Activation('softmax'))
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               2560512   
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 46)                23598     
_________________________________________________________________
activation_5 (Activation)    (None, 46)                0         
Total params: 2,584,110
Trainable params: 2,584,110
Non-trainable params: 0
_________________________________________________________________


In [0]:
model5.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model5.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
250/250 [==============================] - 1s 4ms/step - loss: 1.3336 - accuracy: 0.7120 - val_loss: 0.8529 - val_accuracy: 0.8090
Epoch 2/10
250/250 [==============================] - 1s 4ms/step - loss: 0.5563 - accuracy: 0.8716 - val_loss: 0.7370 - val_accuracy: 0.8300
Epoch 3/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3141 - accuracy: 0.9241 - val_loss: 0.7664 - val_accuracy: 0.8350
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2357 - accuracy: 0.9445 - val_loss: 0.8216 - val_accuracy: 0.8290
Epoch 5/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2159 - accuracy: 0.9473 - val_loss: 0.8601 - val_accuracy: 0.8160
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2052 - accuracy: 0.9506 - val_loss: 0.8457 - val_accuracy: 0.8260
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.1874 - accuracy: 0.9526 - val_loss: 0.8605 - val_accuracy: 0.8190
Epoch 

In [0]:
score = model5.evaluate(x_test, one_hot_test_labels,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

71/71 [==============================] - 0s 2ms/step - loss: 1.0685 - accuracy: 0.7979
Test loss: 1.068505048751831
Test accuracy: 0.7978628873825073


In [0]:
model6 = models.Sequential()
model6.add(layers.Embedding(10000, 64))
model6.add(layers.Bidirectional(tf.keras.layers.LSTM(64)))
model6.add(layers.Dense(64, activation='relu')),
model6.add(layers.Dense(46, activation='softmax'))
model6.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 46)                2990      
Total params: 717,294
Trainable params: 717,294
Non-trainable params: 0
_________________________________________________________________


In [0]:
model6.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model6.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
499/499 [==============================] - 281s 563ms/step - loss: 2.2838 - accuracy: 0.4207 - val_loss: 2.0715 - val_accuracy: 0.4690
Epoch 2/10
499/499 [==============================] - 280s 562ms/step - loss: 2.1127 - accuracy: 0.4485 - val_loss: 2.0805 - val_accuracy: 0.4660
Epoch 3/10
499/499 [==============================] - 280s 561ms/step - loss: 2.1037 - accuracy: 0.4478 - val_loss: 2.0430 - val_accuracy: 0.4650
Epoch 4/10
499/499 [==============================] - 285s 570ms/step - loss: 2.1485 - accuracy: 0.4347 - val_loss: 2.2306 - val_accuracy: 0.4260
Epoch 5/10
499/499 [==============================] - 281s 564ms/step - loss: 2.1874 - accuracy: 0.4427 - val_loss: 2.1359 - val_accuracy: 0.4610
Epoch 6/10
499/499 [==============================] - 287s 575ms/step - loss: 2.1512 - accuracy: 0.4496 - val_loss: 2.1147 - val_accuracy: 0.4680
Epoch 7/10
499/499 [==============================] - 292s 585ms/step - loss: 2.1296 - accuracy: 0.4495 - val_loss: 2.1028 -

In [0]:
model7 = models.Sequential()
model7.add(layers.Embedding(5000, 64))
model7.add(layers.SimpleRNN(64, return_sequences=True))
model7.add(layers.SimpleRNN(64, return_sequences=True))
model7.add(layers.SimpleRNN(64, return_sequences=True))
model7.add(layers.SimpleRNN(64))
model7.add(layers.Dense(46, activation='softmax'))
model7.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dense (Dense)                (None, 46)                2990      
Total params: 356,014
Trainable params: 356,014
Non-trainable params: 0
__________________________________________________

In [0]:
model7.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
history = model7.fit(partial_x_train, partial_y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
